In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from agents.feature.light_rag import insert_text
# insert_text('/Users/kishorkumar/AshaAi/chat-bot-backend/dataset')
# chat_history = [
#     {"role": "human", "content": "Hi there"},
#     {"role": "ai", "content": "Hello! How can I help you?"},
#     {"role": "human", "content": "Tell me about available jobs"},
#     {"role": "ai", "content": "We have job openings in software and data science."}
# ]
# new_query = "Can you check java job openings?"

# inputs = {
#     "user_query": new_query,
#     "chat_history": chat_history,
#     "tool_results": {},  # optional but safe to include
#     "intent": "",        # will be set by extract_intent
#     "response": "",      # will be set by generate_response
# }
# out = core_agent.invoke(inputs)


# out

INFO: Process 2766 Shared-Data created for Single Process
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './knowledge_base/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './knowledge_base/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './knowledge_base/vdb_chunks.json'} 0 data
INFO: Process 2766 initialized updated flags for namespace: [full_docs]
INFO: Process 2766 ready to initialize storage namespace: [full_docs]
INFO: Process 2766 initialized updated flags for namespace: [text_chunks]
INFO: Process 2766 ready to initialize storage namespace: [text_chunks]
INFO: Process 2766 initialized updated flags for namespace: [entities]
INFO: Process 2766 initialized updated flags for namespace: [relationships]
INFO: Process 2766 initialized updated flags for namespace: [chunks]
INFO: Process 2766 initialized updated flags for na

In [ ]:
out

In [ ]:
chat_history = [
    {"role": "human", "content": "Hi there"},
    {"role": "ai", "content": "Hello! How can I help you?"},
    {"role": "human", "content": "Tell me about available jobs"},
    {"role": "ai", "content": "We have job openings in software and data science."},
    {"role": "human", "content": "Can you check java job openings?"},
    {"role": "ai", "content": "I can help you find Java job openings.  To give you the best results, please tell me:\n\n* **Location:** (e.g., San Francisco, CA; New York; Remote)\n* **Experience level:** (e.g., Junior, Mid-level, Senior, Entry-level)* **Specific skills:** (e.g., Spring Boot, Java 8+, Microservices, Cloud Computing)\n* **Desired salary range (optional):**  This will help narrow down the results.\n\nOnce I have this information, I can search for relevant job openings and provide you with a list of matching jobs."},
]
new_query = "just check java job openings?"

inputs = {
    "user_query": new_query,
    "chat_history": [],
    "tool_results": {},  # optional but safe to include
    "intent": "",        # will be set by extract_intent
    "response": "",      # will be set by generate_response
}
for output in core_agent.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(value)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages([
        ("system", '''
        You are an intelligent assistant designed to classify user queries based on their intent. Always respond with only one of the following intents:

            1. feature: When the user is exploring features, offerings or general questions or seeks help on common topics (e.g., how-to, what is, where to find, troubleshooting).
            2. job: When the user asks related to jobs, events, or programs.
            3. profile_update: When the user wants to change or manage their personal profile information(ex: skill, experience, location etc.) or expresses interest in registering, creating an account, or signing up.
            4. unknown: When the user asks a question that does not fit into any of the above categories.

            Carefully analyze the user's query and return the most suitable intent as a single lowercase string (e.g., job).
        '''),
        # ("human", "{user_query}"),
    ])
prompt.append(HumanMessage(content="past content"))
prompt.format_messages(content="latest msg")

In [ ]:
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="in-en", max_results=20)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="json")
print(search.run("site:indeed.com OR site:linkedin.com java jobs"))

In [ ]:
pip install firecrawl-py

In [ ]:
from firecrawl.firecrawl import FirecrawlApp

app = FirecrawlApp(api_key="fc-YOUR_API_KEY",
                   api_url="http//localhost:3002")

# Scrape a website:
scrape_status = app.scrape_url(
  'https://firecrawl.dev', 
  formats=["markdown", "html"]
)
print(scrape_status)

# Crawl a website:
crawl_status = app.crawl_url(
  'https://firecrawl.dev',
  limit=100,
  scrapeOptions={'formats': ['markdown', 'html']},
  poll_interval=30
)
print(crawl_status)

In [ ]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://www.google.com/search?q=java+1+year+experience+job",
        )
        print(result.markdown[:300])  # Show the first 300 characters of extracted text

await main()

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from agents.wesearch_agent.web_search_agent import web_agent

query = "senior python backend engineer jobs in bengalore"
initial_state = {"input": query}

# Call the LangGraph agent
final_state = await web_agent.ainvoke(initial_state)

# Print the final output
print("🔍 Extracted Results:")
for idx, result in enumerate(final_state["output"]):
    print(f"\n--- Result {idx+1} ---\n{result}\n")

# query = "jobs in india"
# jobs = await run_wesearch(query)
# for job in jobs:
#     print(job)